# SSH Setup and Remote Development Lab**Estimated Time:** 30-45 minutes  **Prerequisites:** SSH client installed, access to a remote server (optional)---## Learning ObjectivesBy completing this lab, you will:- ✅ Generate and manage SSH keys- ✅ Configure SSH for convenient access- ✅ Set up VS Code Remote-SSH- ✅ Use SSH port forwarding- ✅ Implement security best practices

---## Exercise 1: Verify SSH InstallationCheck that SSH is installed and working.

In [ ]:
# Check SSH version!ssh -V

In [ ]:
# Check SSH directoryimport osssh_dir = os.path.expanduser('~/.ssh')print(f"SSH directory: {ssh_dir}")print(f"Exists: {os.path.exists(ssh_dir)}")if os.path.exists(ssh_dir):    print(f"\nContents:")    !ls -la ~/.ssh

---## Exercise 2: Generate SSH KeysCreate secure SSH key pairs for authentication.

In [ ]:
# Check for existing keysimport oskey_types = ['id_rsa', 'id_ed25519', 'id_ecdsa']existing_keys = []for key_type in key_types:    key_path = os.path.expanduser(f'~/.ssh/{key_type}')    if os.path.exists(key_path):        existing_keys.append(key_type)        print(f"✅ Found: {key_type}")if not existing_keys:    print("❌ No SSH keys found. Will generate new keys.")else:    print(f"\n⚠️  Existing keys found: {', '.join(existing_keys)}")    print("Consider using existing keys or backup before generating new ones.")

### Generate Ed25519 Key (Recommended)Ed25519 is the modern, secure standard for SSH keys.

In [ ]:
# Generate Ed25519 key (run in terminal)print("To generate an SSH key, run this command in your terminal:")print("\nssh-keygen -t ed25519 -C 'your_email@example.com'")print("\nFollow the prompts:")print("1. Press Enter to accept default location (~/.ssh/id_ed25519)")print("2. Enter a strong passphrase (recommended)")print("3. Confirm the passphrase")

**Note:** We show the command but don't execute it automatically to avoid overwriting existing keys.

In [ ]:
# View your public key (after generation)try:    with open(os.path.expanduser('~/.ssh/id_ed25519.pub'), 'r') as f:        public_key = f.read()    print("Your public key:")    print(public_key)except FileNotFoundError:    print("Public key not found. Generate it first using the command above.")

---## Exercise 3: SSH Configuration FileCreate an SSH config file for convenient remote access.

In [ ]:
import os# Example SSH config contentssh_config_example = """# SSH Configuration File# Location: ~/.ssh/config# Default settings for all hostsHost *    ServerAliveInterval 60    ServerAliveCountMax 3# Example: Personal serverHost myserver    HostName 192.168.1.100    User username    Port 22    IdentityFile ~/.ssh/id_ed25519# Example: Cloud server with different keyHost cloud-gpu    HostName cloud.example.com    User ubuntu    Port 22    IdentityFile ~/.ssh/cloud_key    ForwardAgent yes# Example: Jump host configurationHost internal-server    HostName 10.0.0.50    User admin    ProxyJump myserver"""print(ssh_config_example)print("\nTo use: Save this to ~/.ssh/config")print("Then connect with: ssh myserver")

In [ ]:
# Check if config file existsconfig_path = os.path.expanduser('~/.ssh/config')if os.path.exists(config_path):    print("✅ SSH config file exists")    print("\nCurrent contents:")    with open(config_path, 'r') as f:        print(f.read())else:    print("❌ No SSH config file found")    print("You can create one at: ~/.ssh/config")

---## Exercise 4: SSH AgentUse SSH agent to manage keys and avoid repeated passphrase entry.

In [ ]:
# Check if SSH agent is running!pgrep -l ssh-agent || echo "SSH agent not running"

In [ ]:
# Commands to start SSH agent and add key (run in terminal)print("To use SSH agent, run these commands in your terminal:\n")print("# Start SSH agent")print("eval \"$(ssh-agent -s)\"\n")print("# Add your key")print("ssh-add ~/.ssh/id_ed25519\n")print("# List added keys")print("ssh-add -l")

---## Exercise 5: Testing SSH ConnectionVerify SSH connectivity and troubleshoot issues.

In [ ]:
# Test connection with verbose output (replace with your server)print("To test SSH connection with detailed output:\n")print("ssh -vvv user@hostname\n")print("Verbose flags:")print("-v   : Verbose mode")print("-vv  : More verbose")print("-vvv : Maximum verbosity (debug mode)")

In [ ]:
# Test GitHub SSH connectionprint("Testing GitHub SSH connection...\n")!ssh -T git@github.com 2>&1 || echo "\n(This is expected - GitHub doesn't allow shell access)")

---## Exercise 6: SSH Port ForwardingLearn to access remote services securely via SSH tunnels.

In [ ]:
# Port forwarding examplesexamples = """### Local Port Forwarding (Access remote service locally)# Forward remote Jupyter (8888) to local port 8889ssh -L 8889:localhost:8888 user@remote-server# Then access in browser: http://localhost:8889### Remote Port Forwarding (Expose local service remotely)# Make local service (port 3000) available on remote port 8000ssh -R 8000:localhost:3000 user@remote-server### Dynamic Port Forwarding (SOCKS proxy)# Create a SOCKS proxy on local port 9090ssh -D 9090 user@remote-server### Background Connection# Run SSH tunnel in backgroundssh -fN -L 8889:localhost:8888 user@remote-server#  -f : Go to background#  -N : Don't execute remote command"""print(examples)

---## Exercise 7: VS Code Remote-SSH SetupConfigure VS Code for remote development.

In [ ]:
# VS Code Remote-SSH setup instructionsinstructions = """### VS Code Remote-SSH Setup1. Install Remote-SSH Extension:   - Open VS Code   - Go to Extensions (Ctrl+Shift+X)   - Search for "Remote - SSH"   - Install by Microsoft2. Connect to Remote Server:   - Press F1 or Ctrl+Shift+P   - Type "Remote-SSH: Connect to Host"   - Select or add your server3. Configure SSH Host:   - Press F1 or Ctrl+Shift+P   - Type "Remote-SSH: Open Configuration File"   - Add your server details (see Exercise 3)4. Verify Connection:   - VS Code will install server components   - Terminal will open on remote machine   - File explorer shows remote files### Tips- Use SSH config file for easy connections- Install extensions on remote for full functionality- Port forwarding happens automatically"""print(instructions)

---## Exercise 8: Security Best PracticesVerify SSH security configuration.

In [ ]:
import osimport statdef check_ssh_security():    """Check SSH key permissions and security"""    checks = []        ssh_dir = os.path.expanduser('~/.ssh')        # Check .ssh directory permissions    if os.path.exists(ssh_dir):        dir_stat = os.stat(ssh_dir)        dir_mode = stat.filemode(dir_stat.st_mode)        dir_secure = (dir_stat.st_mode & 0o777) == 0o700        checks.append({            'name': '~/.ssh directory permissions (should be 700)',            'passed': dir_secure,            'value': dir_mode        })        # Check private key permissions    for key_name in ['id_ed25519', 'id_rsa']:        key_path = os.path.join(ssh_dir, key_name)        if os.path.exists(key_path):            key_stat = os.stat(key_path)            key_mode = stat.filemode(key_stat.st_mode)            key_secure = (key_stat.st_mode & 0o777) == 0o600            checks.append({                'name': f'{key_name} permissions (should be 600)',                'passed': key_secure,                'value': key_mode            })        # Check config file permissions    config_path = os.path.join(ssh_dir, 'config')    if os.path.exists(config_path):        config_stat = os.stat(config_path)        config_mode = stat.filemode(config_stat.st_mode)        config_secure = (config_stat.st_mode & 0o777) == 0o600        checks.append({            'name': 'config file permissions (should be 600)',            'passed': config_secure,            'value': config_mode        })        # Print results    print("\n=== SSH Security Check ===")    for check in checks:        status = "✅" if check['passed'] else "❌"        print(f"{status} {check['name']}: {check['value']}")        if checks:        passed = sum(c['passed'] for c in checks)        total = len(checks)        print(f"\n📊 Results: {passed}/{total} checks passed")                if passed < total:            print("\n⚠️  Fix permissions with:")            print("chmod 700 ~/.ssh")            print("chmod 600 ~/.ssh/id_ed25519")            print("chmod 600 ~/.ssh/config")    else:        print("No SSH files found to check.")check_ssh_security()

---## Exercise 9: Common SSH IssuesLearn to troubleshoot common problems.

In [ ]:
troubleshooting = """### Common SSH Issues and Solutions1. Permission Denied (Public Key)   - Check: Public key added to remote ~/.ssh/authorized_keys?   - Check: Private key permissions are 600?   - Check: Using correct key with -i flag?   - Fix: ssh-copy-id user@host2. Connection Timeout   - Check: Correct hostname/IP?   - Check: Firewall blocking port 22?   - Check: Server is running?   - Test: ping hostname3. Host Key Verification Failed   - Cause: Remote host key changed   - Fix: ssh-keygen -R hostname   - Warning: Only do this if you trust the change!4. SSH Agent Not Working   - Start: eval "$(ssh-agent -s)"   - Add key: ssh-add ~/.ssh/id_ed25519   - Verify: ssh-add -l5. Connection Drops   - Add to config:     ServerAliveInterval 60     ServerAliveCountMax 36. Too Many Authentication Failures   - Specify key: ssh -i ~/.ssh/specific_key user@host   - Or in config: IdentitiesOnly yes"""print(troubleshooting)

---## Summary and Key Takeaways### Commands Learned- `ssh-keygen` - Generate SSH keys- `ssh-add` - Add keys to agent- `ssh` - Connect to remote servers- `ssh -L` - Local port forwarding- `ssh -R` - Remote port forwarding- `ssh -D` - Dynamic forwarding (SOCKS)- `ssh-copy-id` - Copy public key to server### Security Best Practices- ✅ Use Ed25519 keys (modern and secure)- ✅ Protect private keys with passphrases- ✅ Set correct permissions (700 for .ssh, 600 for keys)- ✅ Use SSH config for organization- ✅ Use SSH agent to avoid repeated passphrase entry- ✅ Disable password authentication on servers### VS Code Integration- ✅ Install Remote-SSH extension- ✅ Use SSH config for easy connections- ✅ Automatic port forwarding for Jupyter- ✅ Full development environment on remote### Port Forwarding Use Cases- Access remote Jupyter notebooks- Connect to remote databases- Access web applications on remote servers- Secure tunneling for any TCP service### Next Steps- Set up SSH keys on GitHub/GitLab- Configure jump hosts for complex networks- Explore SSH certificates- Learn about SSH tunneling best practices